In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

try: # When on google Colab, let's clone the notebook so we download the cache.
    import google.colab
    repo_path = 'dspy'
    !git -C $repo_path pull origin || git clone https://github.com/stanfordnlp/dspy $repo_path
except:
    repo_path = '.'

if repo_path not in sys.path:
    sys.path.append(repo_path)

# Set up the cache for this notebook
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(repo_path, 'cache')

import pkg_resources # Install the package if it's not installed
if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:
    !pip install -U pip
    !pip install dspy-ai
    !pip install openai~=0.28.1
    # !pip install -e $repo_path

import dspy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Already up to date.


DEFINE A DATASET

In [ ]:
train = [("sarah palin is awesome. | yes she is", "sarah palin", "positive"),
      ("russell brand and katy perry suit each other. they 're both weird.","katy perry", "negative"),
      ("lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you 've hit rock bottom when ...", "lindsay lohan", 'neutral'),
      ('and he has a vision too ! mirzan mahatir is looking like steve jobs right now.', 'steve jobs','neutral'),
      ("i love google ! as a whole with all its services","google","positive")
      ]

# 修改train数据集，使其包含输入`aspect`
train = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in train]

# Check each entry in the dev dataset for errors
dev = [("my 3-year-old was amazed yesterday to find that ' real ' 10 pin bowling is nothing like it is on the wii ...", "wii", "neutral"),
    ("listening 2 '' 3 '' by ms britney spears cant wait 4 the video on oct . 19 you guys need 2 love her new single also buy her single collection nov . 24 !", "britney spears", "positive"),
    ("ff markieai likes animals . hates spiders . hates charlie sheen . likes cookies .", "charlie sheen", "negative"),
    ("espn soccer net news ardiles backs maradona for world cup glory", "maradona", "positive"),
    ("-lsb- happy birthday git * git ! -rsb- ily ! ... some reason im very tired ! but today was really fun ! :-rrb- ugh my ipod died !", "ipod", "neutral"),
    ("hilary swank as amelia earhart is actually pretty great casting.", "hilary swank", "positive"),
    ("video : serena williams gets x-rated for new tennis game ad ! .", "serena williams", "negative"),
    ("lmao i know !! i dnt like harry potter he 's so english !! hehehe its raining here too smdh", "harry potter", "negative"),]
# 修改dev数据集，使其包含输入`aspect`
dev = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in dev]

Define the Metric

In [ ]:
def sentiment_classification_metric(example, pred, trace=None):
    return example.sentiment.lower() == pred.sentiment.lower()

Create the DSPy Context Constrain Program

In [ ]:
import dspy
import openai
from dspy import OpenAI

openai.api_base = "https://api.gpts.vin/v1"
openai.api_key = "sk-FhKeNgB7ZgqvpZqsF357D67Eaa8c42339b02Cf57C56a7d19"

lm = OpenAI(api_base=openai.api_base,
             api_key=openai.api_key,
             model='gpt-3.5-turbo')

dspy.settings.configure(lm=lm)

# Define the DSPy implementation of sentiment polarity analysis with an aspect
class ConstrainSentimentPolarityAnalysis(dspy.Signature):
    """Analyze the sentiment polarity of a given sentence and its aspect."""
    context = dspy.InputField()
    sentence = dspy.InputField(desc="The sentence to analyze.")
    aspect = dspy.InputField()
    sentiment = dspy.OutputField(desc="positive, neutral, or negative.")
class ConstrainSentimentAnalyzer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyze_sentiment = dspy.ChainOfThought(ConstrainSentimentPolarityAnalysis)
    def forward(self, sentence, aspect):
        # Use GPT to create context introduction for the aspect
        context = self.get_constrain_context(sentence, aspect)
        print(f"Context: \n", context)
        prediction = self.analyze_sentiment(sentence=sentence, aspect=aspect,context=context)
        print(f"Prediction:", prediction)
        return dspy.Prediction(context=context, sentiment=prediction.sentiment)

    def get_constrain_context(self, sentence, aspect):
        prompt = (
            f"# Your task is to answer the following questions with 'Yes' or 'No' and input the answer into the slot $PRED$\n"
            f"# From '{sentence}', can we infer that the {aspect} is positive? $PRED$\n"
            f"# From '{sentence}', can we infer that the {aspect} is negative? $PRED$\n"
            f"# From '{sentence}', can we infer that the {aspect} is neutral? $PRED$\n"
        )
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=120,
            n=1,
            stop=None
        )
        context = response.choices[0].message['content'].strip()
        return context
    def analyze_sentiment(self, sentence, aspect, context):
        prompt = (
            f"# Aspect: {aspect}\n"
            f"# Context: {context}\n"
            f"# Sentence: {sentence}\n"
            f"# Please analyze the sentiment polarity (positive, neutral, negative) of the aspect '{aspect}' in the Sentence: \n"
        )
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=50,
            n=1,
            stop=None
        )

        context_sentiment = response.choices[0].message['content'].strip()
        return context_sentiment

Test sentence

In [ ]:
# analyzer = ConstrainSentimentAnalyzer()
# example = {'sentence': 'sarah palin is awesome.', 'aspect': 'sarah palin'}
# result = analyzer(example['sentence'], example['aspect'])
# print(f"Sentence: {example['sentence']}\nAspect: {example['aspect']}\nSentiment: {result.sentiment}")

Evalute metric

In [ ]:
# # Evluate sentiment analysis
# from dspy.evaluate import Evaluate, normalize_text

# def gold_sentiment(example, pred, trace=None):
#     # Normalize the gold standard sentiment and predicted sentiment
#     gold_sent_label = normalize_text(example['sentiment'])
#     generated_sent_label = normalize_text(pred.sentiment)

#     # Check if the predicted sentiment matches the gold standard sentiment
#     return gold_sent_label.lower() == generated_sent_label.lower()

Compile the Program

In [ ]:
from dspy.teleprompt import BootstrapFewShot

# Set up the optimizer
config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)
teleprompter = BootstrapFewShot(metric=sentiment_classification_metric, **config)

# Compile the program
optimized_classifier = teleprompter.compile(ConstrainSentimentAnalyzer(), trainset=train)

dspy/cache/compiler


  0%|          | 0/5 [00:00<?, ?it/s]

Context: 
 From the given statement, "sarah palin is awesome. | yes she is," we can infer that Sarah Palin is positive.


 20%|██        | 1/5 [00:03<00:12,  3.04s/it]

Prediction: Prediction(
    rationale='produce the sentiment. The sentence "sarah palin is awesome" reflects a positive sentiment towards Sarah Palin. The response "yes she is" further reaffirms this positive sentiment.',
    sentiment='positive'
)
Context: 
 # From 'russell brand and katy perry suit each other. they 're both weird.', can we infer that the katy perry is positive? No
# From 'russell brand and katy perry suit each other. they 're both weird.', can we infer that the katy perry is negative? No
# From 'russell brand and katy perry suit each other. they 're both weird.', can we infer that the katy perry is neutral? Yes


...

Prediction: Prediction(
    rationale="produce the sentiment. We see that the statement implies that having fewer followers on Twitter indicates hitting rock bottom. This can be seen as a negative connotation towards Lindsay Lohan's social status and self-worth.",
    sentiment='negative'
)
Context: 
 From the given sentence "and he has a vision too! Mirzan Mahatir is looking like Steve Jobs right now," we cannot infer whether the mention of Steve Jobs is positive, negative, or neutral. Therefore, the answer to all the questions is 'No.'


 80%|████████  | 4/5 [00:16<00:03,  3.86s/it]

Prediction: Prediction(
    rationale='produce the sentiment. We cannot infer the sentiment towards Steve Jobs from this sentence, as it simply states that Mirzan Mahatir is looking like Steve Jobs. Therefore, we cannot determine a specific sentiment towards Steve Jobs in this context.',
    sentiment='neutral'
)
Context: 
 From 'i love google ! as a whole with all its services', can we infer that the google is positive? Yes
From 'i love google ! as a whole with all its services', can we infer that the google is negative? No
From 'i love google ! as a whole with all its services', can we infer that the google is neutral? No


100%|██████████| 5/5 [00:22<00:00,  4.56s/it]

Prediction: Prediction(
    rationale='produce the sentiment. The sentence expresses a positive sentiment towards Google, stating that the speaker loves it as a whole with all its services.',
    sentiment='positive'
)


Evaluate the Program

In [ ]:
from dspy.evaluate import Evaluate

# Set up the evaluator
evaluate = Evaluate(devset=dev, metric=sentiment_classification_metric, num_threads=4, display_progress=True, display_table=0)

# Evaluate the optimized classifier
evaluate(optimized_classifier)

  0%|          | 0/8 [00:00<?, ?it/s]

Context: 
 1. Yes
2. No
3. No
Context: 
 I can't directly access the article mentioned, but based on the information you provided, I can infer that the sentiment towards Maradona in the article is positive. Therefore, the answer for the first question would be "Yes," and for the following two questions would be "No."
Context: 
 # From 'my 3-year-old was amazed yesterday to find that ' real ' 10 pin bowling is nothing like it is on the wii ...', can we infer that the wii is positive? No
# From 'my 3-year-old was amazed yesterday to find that ' real ' 10 pin bowling is nothing like it is on the wii ...', can we infer that the wii is negative? No
# From 'my 3-year-old was amazed yesterday to find that ' real ' 10 pin bowling is nothing like it is on the wii ...',
Context: 
 From the given statement, 'ff markieai likes animals . hates spiders . hates charlie sheen . likes cookies .', we cannot infer the sentiment toward Charlie Sheen.


Average Metric: 1 / 1  (100.0):  12%|█▎        | 1/8 [00:03<00:21,  3.13s/it]

Prediction: Prediction(
    rationale='produce the sentiment. The sentence does not express a specific sentiment towards the Wii. It simply states that real 10-pin bowling is nothing like it is on the Wii.',
    sentiment='neutral'
)


Average Metric: 2 / 3  (66.7):  38%|███▊      | 3/8 [00:04<00:05,  1.08s/it]

...

Average Metric: 5 / 6  (83.3):  75%|███████▌  | 6/8 [00:07<00:01,  1.09it/s]

Prediction: Prediction(
    rationale='produce the sentiment. The sentence reflects a positive sentiment towards Hilary Swank, praising her casting as Amelia Earhart as "pretty great".',
    sentiment='positive'
)
Context: 
 I'm sorry, but I cannot provide a simple 'Yes' or 'No' answer to this question based on the given text. The sentiment towards Harry Potter is not explicitly stated and may not be inferred accurately.


Average Metric: 5 / 8  (62.5): 100%|██████████| 8/8 [00:14<00:00,  1.76s/it]

Prediction: Prediction(
    rationale='Context: Analyzing the sentiment polarity of a given sentence and its aspect.\n\nSentence: lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you \'ve hit rock bottom when ...\n\nAspect: lindsay lohan\n\nReasoning: Let\'s think step by step in order to determine the sentiment. The sentence suggests a comparison where Samantha Ronson has more followers on Twitter, implying a lower social status or popularity for Lindsay Lohan. The phrase "must hate herself" could imply a negative self-perception due to this comparison. However, without explicit negative language directly about Lindsay Lohan and considering the tone, the sentiment towards Lindsay Lohan in this context appears neutral, as it primarily describes a comparative situation rather than directly evaluating her.',
    sentiment='neutral'
)
Prediction: Prediction(
    rationale='produce the sentiment. The sentence "i dnt like harry potter he \'s so engli

62.5

Query the history responses of GPT

In [ ]:
lm.inspect_history(n=1)




Analyze the sentiment polarity of a given sentence and its aspect.

---

Sentence: lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you 've hit rock bottom when ...
Aspect: lindsay lohan
Sentiment: neutral

---

Follow the following format.

Context: ${context}

Sentence: The sentence to analyze.

Aspect: ${aspect}

Reasoning: Let's think step by step in order to ${produce the sentiment}. We ...

Sentiment: positive, neutral, or negative.

---

Context: From the given statement, "sarah palin is awesome. | yes she is," we can infer that Sarah Palin is positive.

Sentence: sarah palin is awesome. | yes she is

Aspect: sarah palin

Reasoning: Let's think step by step in order to produce the sentiment. The sentence "sarah palin is awesome" reflects a positive sentiment towards Sarah Palin. The response "yes she is" further reaffirms this positive sentiment.

Sentiment: positive

---

Context: From the given sentence "and he has a vision too! Mirz

'\n\n\nAnalyze the sentiment polarity of a given sentence and its aspect.\n\n---\n\nSentence: lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you \'ve hit rock bottom when ...\nAspect: lindsay lohan\nSentiment: neutral\n\n---\n\nFollow the following format.\n\nContext: ${context}\n\nSentence: The sentence to analyze.\n\nAspect: ${aspect}\n\nReasoning: Let\'s think step by step in order to ${produce the sentiment}. We ...\n\nSentiment: positive, neutral, or negative.\n\n---\n\nContext: From the given statement, "sarah palin is awesome. | yes she is," we can infer that Sarah Palin is positive.\n\nSentence: sarah palin is awesome. | yes she is\n\nAspect: sarah palin\n\nReasoning: Let\'s think step by step in order to produce the sentiment. The sentence "sarah palin is awesome" reflects a positive sentiment towards Sarah Palin. The response "yes she is" further reaffirms this positive sentiment.\n\nSentiment: positive\n\n---\n\nContext: From the gi